# Wikipedia principal eigenvector
- 그래프의 정점들의 상대적 중요성을 확인하는 방법은 인접 행렬의 주요 고유벡터를 계산하여 정점에 첫 번쨰 고유벡터의 성분 값을 중심성 점수로 할당하는 것
- 위키피디아 문서 내부의 링크 그래프를 분석하여 고유벡터 중심성에 따라 기사를 상대적인 중요도에 따라 순위 매기는 것

In [1]:
import os
from bz2 import BZ2File
from datetime import datetime
from pprint import pprint
from time import time
from urllib.request import urlopen

import numpy as np
from scipy import sparse

from sklearn.decomposition import randomized_svd

## Download data, if not already on disk

In [2]:
redirects_url = "http://downloads.dbpedia.org/3.5.1/en/redirects_en.nt.bz2"
redirects_filename = redirects_url.rsplit("/", 1)[1]

page_links_url = "http://downloads.dbpedia.org/3.5.1/en/page_links_en.nt.bz2"
page_links_filename = page_links_url.rsplit("/", 1)[1]

resources = [
    (redirects_url, redirects_filename),
    (page_links_url, page_links_filename),
]

for url, filename in resources:
    if not os.path.exists(filename):
        print("Downloading data from '%s', please wait..." % url)
        opener = urlopen(url)
        with open(filename, "wb") as f:
            f.write(opener.read())
        print()

## Loading the redirect files

In [3]:
def index(redirects, index_map, k):
    """Find the index of an article name after redirect resolution"""
    k = redirects.get(k, k)
    return index_map.setdefault(k, len(index_map))


DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
SHORTNAME_SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)


def short_name(nt_uri):
    """< 와 > URI 마커 및 공통 URI 접두사를 제거"""
    return nt_uri[SHORTNAME_SLICE]


def get_redirects(redirects_filename):
    """리다이렉션을 구문 분ㅅ거하고 그로부터 추이적으로 닫힌 맵을 구축"""
    redirects = {}
    print("Parsing the NT redirect file")
    for l, line in enumerate(BZ2File(redirects_filename)):
        split = line.split()
        if len(split) != 4:
            print("ignoring malformed line: " + line)
            continue
        redirects[short_name(split[0])] = short_name(split[2])
        if l % 1000000 == 0:
            print("[%s] line: %08d" % (datetime.now().isoformat(), l))

    # 추이적 종결을 계산
    print("Computing the transitive closure of the redirect relation")
    for l, source in enumerate(redirects.keys()):
        transitive_target = None
        target = redirects[source]
        seen = {source}
        while True:
            transitive_target = target
            target = redirects.get(target)
            if target is None or target in seen:
                break
            seen.add(target)
        redirects[source] = transitive_target
        if l % 1000000 == 0:
            print("[%s] line: %08d" % (datetime.now().isoformat(), l))

    return redirects

## Computing the Adjacency matrix

In [4]:
def get_adjacency_matrix(redirects_filename, page_links_filename, limit=None):
    """리다이렉션을 해결한 후, scipy 희소 행렬로서 인접 그래프를 추출

    X는 scipy의 희소 행렬이며, 리다이렉트는 아티클 이름에서 아티클 이름으로 이루어진 파이썬 딕셔너리로 반환
    index_map은 아티클 이름에서 파이썬 int(아티클 인덱스)로 이루어진 파이썬 딕셔너리이다.
    """

    print("Computing the redirect map")
    redirects = get_redirects(redirects_filename)

    print("Computing the integer index map")
    index_map = dict()
    links = list()
    for l, line in enumerate(BZ2File(page_links_filename)):
        split = line.split()
        if len(split) != 4:
            print("ignoring malformed line: " + line)
            continue
        i = index(redirects, index_map, short_name(split[0]))
        j = index(redirects, index_map, short_name(split[2]))
        links.append((i, j))
        if l % 1000000 == 0:
            print("[%s] line: %08d" % (datetime.now().isoformat(), l))

        if limit is not None and l >= limit - 1:
            break

    print("Computing the adjacency matrix")
    X = sparse.lil_matrix((len(index_map), len(index_map)), dtype=np.float32)
    for i, j in links:
        X[i, j] = 1.0
    del links
    print("Converting to CSR representation")
    X = X.tocsr()
    print("CSR conversion done")
    return X, redirects, index_map


# RAM에서 작업이 가능하도록 5백만 개의 링크까지만 처리
X, redirects, index_map = get_adjacency_matrix(
    redirects_filename, page_links_filename, limit=5000000
)
names = {i: name for name, i in index_map.items()}

Computing the redirect map
Parsing the NT redirect file
[2024-01-15T14:06:24.989011] line: 00000000
[2024-01-15T14:06:27.514060] line: 01000000
[2024-01-15T14:06:30.159590] line: 02000000
[2024-01-15T14:06:32.860697] line: 03000000
[2024-01-15T14:06:35.407544] line: 04000000
Computing the transitive closure of the redirect relation
[2024-01-15T14:06:35.611357] line: 00000000
[2024-01-15T14:06:35.948119] line: 01000000
[2024-01-15T14:06:36.307093] line: 02000000
[2024-01-15T14:06:36.691475] line: 03000000
[2024-01-15T14:06:37.125034] line: 04000000
Computing the integer index map
[2024-01-15T14:06:37.164342] line: 00000000
[2024-01-15T14:06:39.612193] line: 01000000
[2024-01-15T14:06:42.146671] line: 02000000
[2024-01-15T14:06:44.651617] line: 03000000
[2024-01-15T14:06:47.193878] line: 04000000
Computing the adjacency matrix
Converting to CSR representation
CSR conversion done


## Computing Principal Singular Vector using Randomized SVD

In [5]:
print("Computing the principal singular vectors using randomized_svd")
t0 = time()
U, s, V = randomized_svd(X, 5, n_iter=3)
print("done in %0.3fs" % (time() - t0))

# 주요 특이벡터의 관련된 가장 강한 연결 요소의 이름을 출력하고 고유벡터와 유사하게 할 수 있다.
# 위키피디아에서 관련된 가장 강한 연결 요소의 이름을 출력
print("Top wikipedia pages according to principal singular vectors")
pprint([names[i] for i in np.abs(U.T[0]).argsort()[-10:]])
pprint([names[i] for i in np.abs(V[0]).argsort()[-10:]])

Computing the principal singular vectors using randomized_svd
done in 1.711s
Top wikipedia pages according to principal singular vectors
[b'1989',
 b'1971',
 b'1975',
 b'1970',
 b'2006',
 b'1972',
 b'1996',
 b'1966',
 b'1967',
 b'2007']
[b'Soviet_Union',
 b'Spain',
 b'Italy',
 b'Canada',
 b'Japan',
 b'Germany',
 b'World_War_II',
 b'France',
 b'United_Kingdom',
 b'United_States']


## Computing Cnetrality scores

In [6]:
def centrality_scores(X, alpha=0.85, max_iter=100, tol=1e-10):
    """Power iteration computation of the principal eigenvector

    This method is also known as Google PageRank and the implementation
    is based on the one from the NetworkX project (BSD licensed too)
    with copyrights by:

      Aric Hagberg <hagberg@lanl.gov>
      Dan Schult <dschult@colgate.edu>
      Pieter Swart <swart@lanl.gov>
    """
    n = X.shape[0]
    X = X.copy()
    incoming_counts = np.asarray(X.sum(axis=1)).ravel()

    print("Normalizing the graph")
    for i in incoming_counts.nonzero()[0]:
        X.data[X.indptr[i] : X.indptr[i + 1]] *= 1.0 / incoming_counts[i]
    dangle = np.asarray(np.where(np.isclose(X.sum(axis=1), 0), 1.0 / n, 0)).ravel()

    scores = np.full(n, 1.0 / n, dtype=np.float32)  # 추정 시작
    for i in range(max_iter):
        print("power iteration #%d" % i)
        prev_scores = scores
        scores = (
            alpha * (scores * X + np.dot(dangle, prev_scores))
            + (1 - alpha) * prev_scores.sum() / n
        )
        # 수렴하는 지 체크: 정규화된 l_inf norm
        scores_max = np.abs(scores).max()
        if scores_max == 0.0:
            scores_max = 1.0
        err = np.abs(scores - prev_scores).max() / scores_max
        print("error: %0.6f" % err)
        if err < n * tol:
            return scores

    return scores


print("Computing principal eigenvector score using a power iteration method")
t0 = time()
scores = centrality_scores(X, max_iter=100)
print("done in %0.3fs" % (time() - t0))
pprint([names[i] for i in np.abs(scores).argsort()[-10:]])

Computing principal eigenvector score using a power iteration method
Normalizing the graph
power iteration #0
error: 0.988448
power iteration #1
error: 0.495766
power iteration #2
error: 0.304177
power iteration #3
error: 0.206611
power iteration #4
error: 0.149243
power iteration #5
error: 0.112217
power iteration #6
error: 0.086737
power iteration #7
error: 0.068371
power iteration #8
error: 0.054681
power iteration #9
error: 0.044212
power iteration #10
error: 0.036058
power iteration #11
error: 0.029603
power iteration #12
error: 0.024433
power iteration #13
error: 0.020259
power iteration #14
error: 0.016847
power iteration #15
error: 0.014051
power iteration #16
error: 0.011746
power iteration #17
error: 0.009838
power iteration #18
error: 0.008256
power iteration #19
error: 0.006936
power iteration #20
error: 0.005830
power iteration #21
error: 0.004912
power iteration #22
error: 0.004139
power iteration #23
error: 0.003489
power iteration #24
error: 0.002945
power iteration #25